In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

# --- **Integrando os dados (Merge)** ---

## 1. Carregamento dos DataFrames tratados


In [2]:
df_foc_queimadas_ro = pd.read_csv('../data/processados/tratados/df_foc_queimadas_ro_2019_2024_20250413.csv')
df_dados_met_ro = pd.read_csv('../data/processados/tratados/df_dados_met_ro_2020_2024_20250413.csv')
df_unid_local_bomb = pd.read_csv('../data/processados/tratados/focos_tratados-df_unid_local_bomb_20250406.csv')

## Função e aplicar nos três dataframes: Padroniza os dados para mitigar problemas de formatação no marge na chave principal que é **município**


In [3]:
import unicodedata

def padroniza_municipio(nome):
    if pd.isnull(nome):
        return nome
    nome = str(nome).strip().lower()  # remove espaços e põe tudo minúsculo
    nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('utf-8')  # remove acentos
    nome = nome.title()  # converte para formato: Porto Velho, Cacoal, etc.
    return nome


In [4]:
df_foc_queimadas_ro['municipio'] = df_foc_queimadas_ro['municipio'].apply(padroniza_municipio)
df_dados_met_ro['municipio'] = df_dados_met_ro['municipio'].apply(padroniza_municipio)
df_unid_local_bomb['municipio'] = df_unid_local_bomb['cidade'].apply(padroniza_municipio)


In [5]:
df_foc_queimadas_ro.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06 17:40:00,Brasil,RONDÔNIA,Rio Crespo,Amazônia
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia


In [6]:
df_dados_met_ro.head()

,data,hora_(utc),temp._ins._(c),temp._max._(c),temp._min._(c),umi._ins._(%),umi._max._(%),umi._min._(%),pto_orvalho_ins._(c),pto_orvalho_max._(c),pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio
0,2023-05-24,2100,29.8,30.9,29.8,67.0,67.0,61.0,22.9,23.4,22.2,997.8,997.8,997.5,0.6,158.0,2.5,713.8,0.0,Porto Velho
1,2023-05-24,2200,27.4,29.8,27.4,80.0,80.0,66.0,23.7,23.8,22.5,998.4,998.4,997.8,0.8,125.0,1.8,57.2,0.0,Porto Velho
2,2023-05-25,1100,24.1,24.1,23.4,95.0,95.0,94.0,23.1,23.1,22.4,1000.5,1000.5,1000.3,0.5,139.0,1.3,102.0,0.0,Porto Velho
3,2023-05-25,1200,25.6,25.6,24.1,92.0,95.0,92.0,24.3,24.3,23.2,1001.0,1001.0,1000.5,0.5,119.0,1.8,547.0,0.0,Porto Velho
4,2023-05-25,1300,27.5,27.6,25.6,79.0,92.0,79.0,23.5,24.2,23.2,1001.3,1001.3,1001.0,1.5,80.0,3.4,1348.0,0.0,Porto Velho


## Normalizar os nomes dos municípios

In [7]:
import unidecode

def padronizar_municipio(nome):
    if pd.isna(nome):
        return ''
    return unidecode.unidecode(nome.strip().upper().replace("'", "").replace("-", ""))


### Aplicando aos dataframes

In [8]:
df_foc_queimadas_ro['municipio_pad'] = df_foc_queimadas_ro['municipio'].apply(padronizar_municipio)
df_dados_met_ro['municipio_pad'] = df_dados_met_ro['municipio'].apply(padronizar_municipio)


In [9]:
focos_municipios = set(df_foc_queimadas_ro['municipio_pad'].unique())
met_municipios = set(df_dados_met_ro['municipio_pad'].unique())

municipios_nao_batem = focos_municipios - met_municipios
print("Municípios nos focos sem dados meteorológicos:", municipios_nao_batem)


Municípios nos focos sem dados meteorológicos: {'MONTE NEGRO', 'CANDEIAS DO JAMARI', 'THEOBROMA', 'GOVERNADOR JORGE TEIXEIRA', 'NOVA UNIAO', 'CASTANHEIRAS', 'OURO PRETO DO OESTE', 'MINISTRO ANDREAZZA', 'PARECIS', 'MIRANTE DA SERRA', 'TEIXEIROPOLIS', 'NOVA MAMORE', 'RIO CRESPO', 'NOVA BRASILANDIA DOESTE', 'PRESIDENTE MEDICI', 'CABIXI', 'VALE DO PARAISO', 'PIMENTA BUENO', 'COSTA MARQUES', 'GUAJARAMIRIM', 'SERINGUEIRAS', 'ALVORADA DOESTE', 'ALTO ALEGRE DOS PARECIS', 'CACAULANDIA', 'NOVO HORIZONTE DO OESTE', 'CHUPINGUAIA', 'ITAPUA DO OESTE', 'MACHADINHO DOESTE', 'URUPA', 'JIPARANA', 'PIMENTEIRAS DO OESTE', 'ROLIM DE MOURA', 'JARU', 'ESPIGAO DOESTE', 'SAO FRANCISCO DO GUAPORE', 'CUJUBIM', 'SAO FELIPE DOESTE', 'CAMPO NOVO DE RONDONIA', 'ALTA FLORESTA DOESTE', 'PRIMAVERA DE RONDONIA', 'BURITIS', 'VALE DO ANARI', 'CEREJEIRAS', 'SANTA LUZIA DOESTE', 'CORUMBIARA', 'ALTO PARAISO', 'SAO MIGUEL DO GUAPORE', 'COLORADO DO OESTE'}


### Municípios sem dados meteorológicos

In [10]:
pd.Series(list(municipios_nao_batem)).to_csv("../data/processados/municipios_sem_dados_meteorologicos.csv", index=False)


## 2. Renomeação de colunas meteorológicas


In [11]:
df_dados_met_ro.rename(columns={
    'Temp. Ins. (C)': 'temp_inst',
    'Temp. Max. (C)': 'temp_max',
    'Temp. Min. (C)': 'temp_min',
    'Umi. Ins. (%)': 'umid_inst',
    'Umi. Max. (%)': 'umid_max',
    'Umi. Min. (%)': 'umid_min',
    'Orvalho Ins. (C)': 'orvalho',
    'Ptc Máx. (C)': 'ptc_max',
    'Pressão Ins. (hPa)': 'pressao_inst',
    'Pressão Máx. (hPa)': 'pressao_max',
    'Pressão Mín. (hPa)': 'pressao_min',
    'Vel. Vento (m/s)': 'vento_vel',
    'Dir. Vento (graus)': 'vento_dir',
    'Raj. Vento (m/s)': 'vento_raj',
    'Radiação (KJ/m²)': 'radiacao',
    'Chuva (mm)': 'chuva',
    'Data (UTC)': 'data',
    'município': 'municipio'
}, inplace=True)


In [12]:
df_dados_met_ro.head()

,data,hora_(utc),temp._ins._(c),temp._max._(c),temp._min._(c),umi._ins._(%),umi._max._(%),umi._min._(%),pto_orvalho_ins._(c),pto_orvalho_max._(c),...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio,municipio_pad
0,2023-05-24,2100,29.8,30.9,29.8,67.0,67.0,61.0,22.9,23.4,...,997.8,997.8,997.5,0.6,158.0,2.5,713.8,0.0,Porto Velho,PORTO VELHO
1,2023-05-24,2200,27.4,29.8,27.4,80.0,80.0,66.0,23.7,23.8,...,998.4,998.4,997.8,0.8,125.0,1.8,57.2,0.0,Porto Velho,PORTO VELHO
2,2023-05-25,1100,24.1,24.1,23.4,95.0,95.0,94.0,23.1,23.1,...,1000.5,1000.5,1000.3,0.5,139.0,1.3,102.0,0.0,Porto Velho,PORTO VELHO
3,2023-05-25,1200,25.6,25.6,24.1,92.0,95.0,92.0,24.3,24.3,...,1001.0,1001.0,1000.5,0.5,119.0,1.8,547.0,0.0,Porto Velho,PORTO VELHO
4,2023-05-25,1300,27.5,27.6,25.6,79.0,92.0,79.0,23.5,24.2,...,1001.3,1001.3,1001.0,1.5,80.0,3.4,1348.0,0.0,Porto Velho,PORTO VELHO


In [13]:
print(df_dados_met_ro.shape[0])

28419


## 📆 Tratamento das datas

In [14]:
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'], errors='coerce')
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'], errors='coerce')

### Padronizando as datas para o merge formato YYYY-MM-DD

In [15]:
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data']).dt.date
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data']).dt.date


### Merge entre Focos de Queimada e Dados Meteorológicos

___

## Nova coluna data_formatada para compor o Marge

In [16]:
df_foc_queimadas_ro['data_formatada'] = pd.to_datetime(df_foc_queimadas_ro['data']).dt.date
df_dados_met_ro['data_formatada'] = pd.to_datetime(df_dados_met_ro['data']).dt.date


In [17]:
# Até que enfim cheguei aqui...😩😩🤯
# Merge pela alma do projeto: data + município
df_merge = pd.merge(
    df_foc_queimadas_ro,
    df_dados_met_ro,
    left_on=['data_formatada', 'municipio'],
    right_on=['data_formatada', 'municipio'],
    how='left'
)

df_merge.head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06,Brasil,RONDÔNIA,Rio Crespo,Amazônia,RIO CRESPO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_completos = df_merge.dropna()
df_completos.head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


## Lista apenas colunas com dados meteorológicos

In [19]:
# Lista das colunas meteorológicas
colunas_meteorologicas = [
    'hora_(utc)', 'temp._ins._(c)', 'temp._max._(c)', 'temp._min._(c)',
    'umi._ins._(%)', 'umi._max._(%)', 'umi._min._(%)',
    'pto_orvalho_ins._(c)', 'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)',
    'pressao_ins._(hpa)', 'pressao_max._(hpa)', 'pressao_min._(hpa)',
    'vel._vento_(m/s)', 'dir._vento_(m/s)', 'raj._vento_(m/s)',
    'radiacao_(kj/m²)', 'chuva_(mm)'
]

df_base_modelo_incendios  = df_merge.dropna(subset=colunas_meteorologicas)


In [20]:
df_base_modelo_incendios .head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


In [21]:
print(df_base_modelo_incendios .shape[0])

61997


## ----  Validação Pós-Merge ----

### 1. Verificar dados ausentes (NaNs)

In [22]:
# Verificar quantidade de valores ausentes por coluna
df_base_modelo_incendios.isna().sum().sort_values(ascending=False)


id_bdq                  0
umi._ins._(%)           0
chuva_(mm)              0
radiacao_(kj/m²)        0
raj._vento_(m/s)        0
dir._vento_(m/s)        0
vel._vento_(m/s)        0
pressao_min._(hpa)      0
pressao_max._(hpa)      0
pressao_ins._(hpa)      0
pto_orvalho_min._(c)    0
pto_orvalho_max._(c)    0
pto_orvalho_ins._(c)    0
umi._min._(%)           0
umi._max._(%)           0
temp._min._(c)          0
foco_id                 0
temp._max._(c)          0
temp._ins._(c)          0
hora_(utc)              0
data_y                  0
data_formatada          0
municipio_pad_x         0
bioma                   0
municipio               0
estado                  0
pais                    0
data_x                  0
lon                     0
lat                     0
municipio_pad_y         0
dtype: int64

### 2. Verificar se existem linhas duplicadas

In [23]:
# Verifica se há registros duplicados completos
duplicados = df_base_modelo_incendios.duplicated()
print(f"Total de linhas duplicadas: {duplicados.sum()}")


Total de linhas duplicadas: 0


## Renomeei a data de novo aqui 😅

In [28]:
# Renomear a coluna data_x para data
df_base_modelo_incendios.rename(columns={"data_x": "data"}, inplace=True)


C:\Users\ander\AppData\Local\Temp\ipykernel_19880\3629414703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_modelo_incendios.rename(columns={"data_x": "data"}, inplace=True)


In [29]:
df_base_modelo_incendios.head()  # Exibir as primeiras 10 linhas do DataFrame

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


### 3. Verificar formato de colunas chave

In [31]:
df_base_modelo_incendios['data'] = pd.to_datetime(df_base_modelo_incendios['data'], errors='coerce')
print(df_base_modelo_incendios['data'].dtypes)


datetime64[ns]


C:\Users\ander\AppData\Local\Temp\ipykernel_19880\817531445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_modelo_incendios['data'] = pd.to_datetime(df_base_modelo_incendios['data'], errors='coerce')


### 4. Verificar se há dados de múltiplos anos, municípios e datas esperadas

In [32]:
# Quantidade de municípios
print("Municípios únicos:", df_base_modelo_incendios['municipio'].nunique())

# Amostragem de anos
print("Anos na base:", df_base_modelo_incendios['data'].dt.year.unique())

# Checar se há dados para todos os anos esperados (ex: 2019 a 2024)
anos_esperados = set(range(2019, 2025))
anos_presentes = set(df_base_modelo_incendios['data'].dt.year.unique())
print("Anos ausentes:", anos_esperados - anos_presentes)


Municípios únicos: 4
Anos na base: [2020 2023 2024]
Anos ausentes: {2019, 2021, 2022}


# ---- DEPOIS DAQUI, PERCEBI QUE ESSE MERGE NÃO IRIA ROLAR, POIS OS DADOS DE FOCOS NÃO ESTÃO SE CONCILIANDO CORRETAMENTE COM OS DADOS METEOROLÓGICOS

# --- **MUDANÇA DE PLANOS COM O (Merge) A PARTIR DAQUI** ---

### 🧩 PLANO DE AÇÃO: Agrupar dados meteorológicos por município de referência

Vamos assumir que alguns municípios vizinhos podem usar os dados meteorológicos da estação mais próxima.

## Mapeamento da proximidade geográfica e facilidade logística

## 1. Criar uma coluna "estação de referência" para os municípios sem dados meteorológicos

In [42]:
mapeamento_estacoes = {
    # Região Central (Proximidade: Cacoal)
    'Ji-Parana': 'Cacoal',
    'Pimenta Bueno': 'Cacoal',
    'Rolim De Moura': 'Cacoal',
    'Alta Floresta D\'Oeste': 'Cacoal',
    'Santa Luzia D\'Oeste': 'Cacoal',
    'Espigao D\'Oeste': 'Cacoal',
    'Ministro Andreazza': 'Cacoal',
    'Parecis': 'Cacoal',
    'Primavera De Rondonia': 'Cacoal',
    'Novo Horizonte Do Oeste': 'Cacoal',
    'Castanheiras': 'Cacoal',
    'Sao Felipe D\'Oeste': 'Cacoal',
    
    # Região Norte (Proximidade: Ariquemes)
    'Ariquemes': 'Ariquemes',  # já é base
    'Jaru': 'Ariquemes',
    'Cujubim': 'Ariquemes',
    'Buritis': 'Ariquemes',
    'Monte Negro': 'Ariquemes',
    'Machadinho D\'Oeste': 'Ariquemes',
    'Campo Novo De Rondonia': 'Ariquemes',
    'Cacaulandia': 'Ariquemes',
    'Theobroma': 'Ariquemes',
    'Vale Do Anari': 'Ariquemes',
    'Vale Do Paraiso': 'Ariquemes',
    'Rio Crespo': 'Ariquemes',
    'Nova Uniao': 'Ariquemes',
    'Governador Jorge Teixeira': 'Ariquemes',
    
    # Região Metropolitana (Porto Velho)
    'Porto Velho': 'Porto Velho',  # já é base
    'Guajara-Mirim': 'Porto Velho',
    'Candeias Do Jamari': 'Porto Velho',
    'Itapua Do Oeste': 'Porto Velho',
    
    # Extremo Oeste (Proximidade: Porto Velho)
    'Costa Marques': 'Porto Velho',
    'Sao Francisco Do Guapore': 'Porto Velho',
    'Sao Miguel Do Guapore': 'Porto Velho',
    'Seringueiras': 'Porto Velho',
    'Alvorada D\'Oeste': 'Porto Velho',
    
    # Extremo Sul (Proximidade: Vilhena)
    'Vilhena': 'Vilhena',  # já é base
    'Colorado Do Oeste': 'Vilhena',
    'Cerejeiras': 'Vilhena',
    'Cabixi': 'Vilhena',
    'Pimenteiras Do Oeste': 'Vilhena',
    'Corumbiara': 'Vilhena',
    'Chupinguaia': 'Vilhena',
    'Alto Alegre Dos Parecis': 'Vilhena',
    
    # Região Centro-Norte/Sul (ajuste por contexto)
    'Presidente Medici': 'Cacoal',
    'Ouro Preto Do Oeste': 'Cacoal',
    'Nova Brasilandia D\'Oeste': 'Cacoal',
    'Urupa': 'Cacoal',
    'Mirante Da Serra': 'Cacoal',
    'Alto Paraiso': 'Ariquemes',
}


## 2. Nova coluna no dataset de focos

### Preparar df_dados_met_ro para o merge

In [43]:

df_dados_met_ro.rename(columns={'municipio': 'estacao_ref', 'Data (UTC)': 'data'}, inplace=True)

# Converter datas
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'])
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'])


Esse código abaixo é muito show! Preciso usar ele para uma outra ideia

In [44]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)


In [46]:
df_foc_queimadas_ro.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad,data_formatada,estacao_ref
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06,Brasil,RONDÔNIA,Rio Crespo,Amazônia,RIO CRESPO,2019-11-06,Ariquemes
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes


## 3. Fazer o merge com base em estacao_ref + data (Agora sim 😎✅🎲)

In [47]:
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'])
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'])

df_dados_met_ro.rename(columns={'municipio': 'estacao_ref'}, inplace=True)

df_base_modelo_incendios = pd.merge(
    df_foc_queimadas_ro,
    df_dados_met_ro,
    on=['estacao_ref', 'data'],
    how='inner'  # ou 'left' se quiser manter todos os focos e só pegar meteorologia onde houver
)



### Replicação do Mapeamento

In [50]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)


### Atualizar o mapeamento_estacoes para os que faltam

In [54]:
# Atualizando o mapeamento com os casos restantes
mapeamento_estacoes.update({
    'Nova Mamore': 'Porto Velho',     # Município vizinho, proximidade geográfica com estação PVH
    'Cacoal': 'Cacoal',               # Aqui é fácil, já existe a estação em Cacoal!
    'Teixeiropolis': 'Ariquemes',     # Fica ali na região central-norte, mais próximo de Ariquemes
})


In [55]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)

### checar os que ficaram de fora

## Esse código abaixo é que determina a vitória do merge. Se realmente o merge deu certo!!

In [56]:
faltantes = df_foc_queimadas_ro[df_foc_queimadas_ro['estacao_ref'].isna()]
print(f"Focos SEM estação meteorológica vinculada: {faltantes['municipio'].unique()}")


Focos SEM estação meteorológica vinculada: []


In [57]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y,data_formatada_y
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,992.1,991.6,1.2,152.0,3.6,96.9,0.0,CACOAL,2020-06-19
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.8,992.1,1.4,139.0,2.6,713.0,0.0,CACOAL,2020-06-19
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,CACOAL,2020-06-19
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,CACOAL,2020-06-19
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.6,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,CACOAL,2020-06-19


In [59]:
print(df_base_modelo_incendios.shape[0])  # Total de registros após o merge

150008


In [63]:
df_base_modelo_incendios.iloc[38234:38238]

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y,data_formatada_y
38234,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,944.7,945.9,944.7,2.0,304.0,5.6,3258.2,0.0,VILHENA,2023-10-24
38235,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,943.4,944.6,943.4,2.0,339.0,5.5,2933.1,0.0,VILHENA,2023-10-24
38236,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,942.8,943.4,942.8,2.3,330.0,5.1,1866.0,0.0,VILHENA,2023-10-24
38237,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,941.8,942.8,941.8,1.7,328.0,5.1,1384.4,0.0,VILHENA,2023-10-24


## 🔧 Etapa de limpeza pós-merge 

In [72]:

df_base_modelo_incendios = df_base_modelo_incendios.rename(columns={
    'municipio_pad_y': 'estacao_ref'
})


### Remover a coluna data do merge anterior

In [77]:
# Renomear a coluna data 
df_base_modelo_incendios = df_base_modelo_incendios.rename(columns={'data_formatada': 'data'})

In [78]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),estacao_ref,data
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,992.1,991.6,1.2,152.0,3.6,96.9,0.0,CACOAL,2020-06-19
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.8,992.1,1.4,139.0,2.6,713.0,0.0,CACOAL,2020-06-19
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,CACOAL,2020-06-19
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,CACOAL,2020-06-19
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.6,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,CACOAL,2020-06-19


## --- 🧪 **Verifca se os dados meteorológicos estão consistentes por data e estação: ** ---

In [80]:
df_base_modelo_incendios[['data', 'municipio', 'estacao_ref', 'chuva_(mm)', 'temp._ins._(c)', 'umi._ins._(%)']]


,data,data,municipio,estacao_ref,estacao_ref,chuva_(mm),temp._ins._(c),umi._ins._(%)
0,2020-06-19,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,20.0,90.0
1,2020-06-19,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,22.7,82.0
2,2020-06-19,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,26.1,69.0
3,2020-06-19,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,30.1,54.0
4,2020-06-19,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,31.4,45.0
...,...,...,...,...,...,...,...,...
150003,2024-12-30,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,29.6,69.0
150004,2024-12-30,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,31.5,61.0
150005,2024-12-30,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,31.5,59.0
150006,2024-12-30,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,32.1,58.0


## Aqui eu trago apenas os dados com chuvaa maior que 0, ou seja, com chuva. 🌧

In [82]:
# Filtrar os dados onde a coluna 'chuva_(mm)' é maior que 0
dados_com_chuva = df_base_modelo_incendios[df_base_modelo_incendios['chuva_(mm)'] > 0]

# Exibir os primeiros registros do resultado
dados_com_chuva.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),estacao_ref,data
32,1521156494,158e8837-a683-3d2c-9c94-888072ef186f,-11.230,-60.828,2020-12-03,Brasil,RONDÔNIA,Espigao D'Oeste,Amazônia,ESPIGAO DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
58,1521156574,e1cf038a-3f45-3faf-a299-3a1b93744060,-11.136,-61.558,2020-12-03,Brasil,RONDÔNIA,Ministro Andreazza,Amazônia,MINISTRO ANDREAZZA,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
71,1521156532,fbe55e32-a0dd-30f1-8236-ce2288cd1dae,-11.284,-60.992,2020-12-03,Brasil,RONDÔNIA,Espigao D'Oeste,Amazônia,ESPIGAO DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
84,1521156471,37ad6f07-c6a2-3a29-aa30-c860484cf88a,-11.879,-61.179,2020-12-03,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
97,1521156745,9877826f-c459-3546-b58b-6061842a0d72,-11.486,-62.130,2020-12-03,Brasil,RONDÔNIA,Nova Brasilandia D'Oeste,Amazônia,NOVA BRASILANDIA DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03


In [83]:
print(dados_com_chuva.shape[0])

2388
